In [1]:
from pyscf import gto, lib
import numpy as np
import itertools

np.set_printoptions(16)

In [2]:
lib.num_threads(32)

32

In [3]:
mol = gto.Mole(atom="""
C          0.89735       -0.00113        0.08382
C          2.41558       -0.00973        0.08398
C          2.95403       -1.39223        0.44060
C          4.47840       -1.40490        0.43927
C          5.00472       -2.79081        0.79800
C          6.52868       -2.80677        0.79924
C          7.04947       -4.19283        1.15712
C          8.57098       -4.20695        1.16283
C          9.09164       -5.59214        1.51883
C         10.61579       -5.60738        1.52100
C         11.13683       -6.99323        1.87886
C         12.66149       -7.01081        1.88281
C         13.18196       -8.39866        2.24144
C         14.70681       -8.42700        2.24831
C         15.22338       -9.80961        2.60521
H          0.52316        0.99486       -0.17283
H          0.50536       -0.27018        1.06991
H          0.50315       -0.71346       -0.64803
H          2.78364        0.73060        0.80387
H          2.78126        0.28908       -0.90547
H          2.57994       -2.13111       -0.27881
H          2.58434       -1.68881        1.42971
H          4.85726       -0.66903        1.15930
H          4.85232       -1.11324       -0.54954
H          4.62816       -3.52793        0.07814
H          4.63163       -3.08460        1.78691
H          6.90615       -2.07036        1.51945
H          6.90302       -2.51453       -0.18957
H          6.67226       -4.92766        0.43715
H          6.67377       -4.48532        2.14643
H          8.94855       -3.47382        1.88410
H          8.94922       -3.91225        0.17366
H          8.71508       -6.32967        0.79715
H          8.71691       -5.88744        2.50652
H         10.99100       -4.87238        2.24175
H         10.98995       -5.31412        0.53343
H         10.75962       -7.72809        1.15673
H         10.76022       -7.28664        2.86631
H         13.03725       -6.27521        2.60442
H         13.03678       -6.71689        0.89495
H         12.80104       -9.13172        1.51944
H         12.80163       -8.68997        3.22825
H         15.08995       -7.69771        2.97148
H         15.08954       -8.13915        1.26219
H         14.87740      -10.11040        3.59946
H         16.31717       -9.81702        2.60682
H         14.87685      -10.55420        1.88142
""", basis="def2-TZVP").build()

In [4]:
nshl = mol.nbas2, 1
nao = mol.nao
nao, nshl

(657, 293)

## test_int3c2e_s1_full

In [5]:
%%time
out = mol.intor("int3c2e")

CPU times: user 8.23 s, sys: 1.51 s, total: 9.74 s
Wall time: 317 ms


In [6]:
out_c = out.transpose(2, 1, 0)
assert out_c.flags.c_contiguous
scale = np.linspace(-1, 1, out.size)
print(out_c.sum())
print((out_c.flatten() * scale).sum())

481358.8444863676
127300.77647158667


## int3c2e_s1_slice

In [7]:
%%time
out = mol.intor("int3c2e", shls_slice=(7, 275, 12, 129, 5, 264))

CPU times: user 3.53 s, sys: 978 ms, total: 4.5 s
Wall time: 149 ms


In [8]:
out_c = out.transpose(2, 1, 0)
assert out_c.flags.c_contiguous
scale = np.linspace(-1, 1, out.size)
print(out_c.sum())
print((out_c.flatten() * scale).sum())

213342.8408809482
55811.71336929558


## int3c2e_ip1_s1_slice

In [9]:
%%time
out = mol.intor("int3c2e_ip1", shls_slice=(7, 275, 12, 129, 5, 264))

CPU times: user 8.34 s, sys: 2.94 s, total: 11.3 s
Wall time: 383 ms


In [10]:
out_c = out.transpose(0, 3, 2, 1)
assert out_c.flags.c_contiguous
scale = np.linspace(-1, 1, out.size)
print(out_c.sum())
print((out_c.flatten() * scale).sum())

-10007.110461088905
1698.1730005702707


## int2c2e_ip1_s1_slice

In [11]:
%%time
out = mol.intor("int2c2e_ip1", shls_slice=(7, 275, 5, 264))

CPU times: user 98.8 ms, sys: 10.1 ms, total: 109 ms
Wall time: 4.19 ms


In [12]:
out_c = out.transpose(0, 2, 1)
assert out_c.flags.c_contiguous
scale = np.linspace(-1, 1, out.size)
print(out_c.sum())
print((out_c.flatten() * scale).sum())

-693.0183096938822
1960.186871018245


## int2c2e_ip1ip2_s1_slice

In [13]:
%%time
out = mol.intor("int2e_ip1ip2", shls_slice=(7, 30, 5, 52, 58, 89, 125, 156))

CPU times: user 14.7 s, sys: 2.26 s, total: 17 s
Wall time: 540 ms


In [14]:
out_c = out.transpose(0, 4, 3, 2, 1).copy()
assert out_c.flags.c_contiguous
scale = np.linspace(-1, 1, out.size)
print(out_c.sum())
print((out_c.flatten() * scale).sum())

0.06946888260914107
-0.021085092464779066


## int3c2e_s2ij_full

In [15]:
%%time
out = mol.intor("int3c2e", aosym="s2ij")

CPU times: user 4.49 s, sys: 1.73 s, total: 6.22 s
Wall time: 206 ms


In [16]:
out_c = out.transpose(1, 0)
assert out_c.flags.c_contiguous
scale = np.linspace(-1, 1, out.size)
print(out_c.sum())
print((out_c.flatten() * scale).sum())

311401.69988397433
84387.58117014868


## int3c2e_ip2_s2ij_slice

In [17]:
%%time
out = mol.intor("int3c2e_ip2", shls_slice=(0, 275, 0, 275, 7, 264), aosym="s2ij")

CPU times: user 7.59 s, sys: 4.09 s, total: 11.7 s
Wall time: 384 ms


In [18]:
out_c = out.transpose(0, 2, 1)
assert out_c.flags.c_contiguous
scale = np.linspace(-1, 1, out.size)
print(out_c.sum())
print((out_c.flatten() * scale).sum())

10545.286228548592
-1645.065493306819


**Note** In pyscf, it seems that `s2ij` in `getints3c`, starting shell should always zero, otherwise it may not seems to be correct.

## int2e_ip2_s2ij_slice

In [19]:
%%time
out = mol.intor("int2e_ip2", shls_slice=(10, 50, 10, 50, 127, 168, 215, 272), aosym="s2ij")

CPU times: user 5.45 s, sys: 412 ms, total: 5.86 s
Wall time: 200 ms


In [20]:
out_c = out.transpose(0, 3, 2, 1).copy()
assert out_c.flags.c_contiguous
scale = np.linspace(-1, 1, out.size)
print(out_c.sum())
print((out_c.flatten() * scale).sum())

84.37051265850292
9.624874473531595


## int2c2e_s2ij_slice

In [21]:
%%time
out = mol.intor("int2c2e", shls_slice=(10, 283, 10, 283), aosym="s2ij")

CPU times: user 159 ms, sys: 3.99 ms, total: 163 ms
Wall time: 6 ms


In [22]:
out_c = out[np.tril_indices(out.shape[0])]
assert out_c.flags.c_contiguous
scale = np.linspace(-1, 1, out_c.size)
print(out_c.sum())
print((out_c.flatten() * scale).sum())

168170.6575820652
50272.65108549222


## Molecule information

In [23]:
np.set_printoptions(threshold=1e15)

In [24]:
mol._atm

array([[  6,  20,   1,  23,   0,   0],
       [  6,  24,   1,  27,   0,   0],
       [  6,  28,   1,  31,   0,   0],
       [  6,  32,   1,  35,   0,   0],
       [  6,  36,   1,  39,   0,   0],
       [  6,  40,   1,  43,   0,   0],
       [  6,  44,   1,  47,   0,   0],
       [  6,  48,   1,  51,   0,   0],
       [  6,  52,   1,  55,   0,   0],
       [  6,  56,   1,  59,   0,   0],
       [  6,  60,   1,  63,   0,   0],
       [  6,  64,   1,  67,   0,   0],
       [  6,  68,   1,  71,   0,   0],
       [  6,  72,   1,  75,   0,   0],
       [  6,  76,   1,  79,   0,   0],
       [  1,  80,   1,  83,   0,   0],
       [  1,  84,   1,  87,   0,   0],
       [  1,  88,   1,  91,   0,   0],
       [  1,  92,   1,  95,   0,   0],
       [  1,  96,   1,  99,   0,   0],
       [  1, 100,   1, 103,   0,   0],
       [  1, 104,   1, 107,   0,   0],
       [  1, 108,   1, 111,   0,   0],
       [  1, 112,   1, 115,   0,   0],
       [  1, 116,   1, 119,   0,   0],
       [  1, 120,   1, 12

In [25]:
mol._bas

array([[  0,   0,   6,   1,   0, 208, 214,   0],
       [  0,   0,   2,   1,   0, 220, 222,   0],
       [  0,   0,   1,   1,   0, 224, 225,   0],
       [  0,   0,   1,   1,   0, 226, 227,   0],
       [  0,   0,   1,   1,   0, 228, 229,   0],
       [  0,   1,   4,   1,   0, 230, 234,   0],
       [  0,   1,   1,   1,   0, 238, 239,   0],
       [  0,   1,   1,   1,   0, 240, 241,   0],
       [  0,   2,   1,   1,   0, 242, 243,   0],
       [  0,   2,   1,   1,   0, 244, 245,   0],
       [  0,   3,   1,   1,   0, 246, 247,   0],
       [  1,   0,   6,   1,   0, 208, 214,   0],
       [  1,   0,   2,   1,   0, 220, 222,   0],
       [  1,   0,   1,   1,   0, 224, 225,   0],
       [  1,   0,   1,   1,   0, 226, 227,   0],
       [  1,   0,   1,   1,   0, 228, 229,   0],
       [  1,   1,   4,   1,   0, 230, 234,   0],
       [  1,   1,   1,   1,   0, 238, 239,   0],
       [  1,   1,   1,   1,   0, 240, 241,   0],
       [  1,   2,   1,   1,   0, 242, 243,   0],
       [  1,   2,   

In [26]:
mol._env

array([ 0.0000000000000000e+00,  0.0000000000000000e+00,
        0.0000000000000000e+00,  0.0000000000000000e+00,
        0.0000000000000000e+00,  0.0000000000000000e+00,
        0.0000000000000000e+00,  0.0000000000000000e+00,
        0.0000000000000000e+00,  0.0000000000000000e+00,
        0.0000000000000000e+00,  0.0000000000000000e+00,
        0.0000000000000000e+00,  0.0000000000000000e+00,
        0.0000000000000000e+00,  0.0000000000000000e+00,
        0.0000000000000000e+00,  0.0000000000000000e+00,
        0.0000000000000000e+00,  0.0000000000000000e+00,
        1.6957457378784582e+00, -2.1353905207585197e-03,
        1.5839684376104349e-01,  0.0000000000000000e+00,
        4.5647846319768721e+00, -1.8387035192018054e-02,
        1.5869919994097390e-01,  0.0000000000000000e+00,
        5.5823076637489297e+00, -2.6309334024032163e+00,
        8.3261333048336628e-01,  0.0000000000000000e+00,
        8.4629494762521720e+00, -2.6548762324014556e+00,
        8.3009999473769469e-01,

In [27]:
! lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         48 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  32
  On-line CPU(s) list:   0-31
Vendor ID:               AuthenticAMD
  Model name:            AMD Ryzen 9 7945HX with Radeon Graphics
    CPU family:          25
    Model:               97
    Thread(s) per core:  2
    Core(s) per socket:  16
    Socket(s):           1
    Stepping:            2
    Frequency boost:     enabled
    CPU max MHz:         5461.0000
    CPU min MHz:         400.0000
    BogoMIPS:            4990.83
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mc
                         a cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall n
                         x mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_go
                         od amd_lbr_v2 nopl nonstop_tsc cpuid extd_apicid aperfm
                         perf rapl pni pclmul